In [1]:
import json
import csv
#import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
#import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Quora/BITSPQuora.csv')


In [3]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers
0,0,Birla-Institute-of-Technology-and-Science-Pila...,What made BITS Pilani so good? How did it mana...,2,2,2/2,No,Followed,2020-07-24,https://www.quora.com/What-made-BITS-Pilani-so...,Thanks for A2A What makes a college good in ge...
1,1,Birla-Institute-of-Technology-and-Science-Pila...,What made BITS Pilani so good? How did it mana...,2,2,2/2,No,Followed,2020-07-24,https://www.quora.com/What-made-BITS-Pilani-so...,"Andhon mein kana, Raja ! This is the proverb t..."
2,2,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better, BITS Pilani Microelectronics ...",5,2,5/2,No,Followed,2020-07-20,https://www.quora.com/Which-is-better-BITS-Pil...,This correct answer is in Indian context: Sinc...
3,3,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better, BITS Pilani Microelectronics ...",5,2,5/2,No,Followed,2020-07-20,https://www.quora.com/Which-is-better-BITS-Pil...,Obviously BITS Microelectronics. BITS is the f...
4,4,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better to get into, BITS Pilani DUBAI...",1,1,1/1,No,Followed,2020-07-19,https://www.quora.com/Which-is-better-to-get-i...,It's absolute foolishness to compare IITs with...
...,...,...,...,...,...,...,...,...,...,...,...
2030,2030,Birla-Institute-of-Technology-and-Science-Pila...,Does it even make sense to expect any drop in ...,30,2,30/2,Yes,Followed,2018-07-26,https://www.quora.com/Does-it-even-make-sense-...,"Thanks for A2A!!!! Well, You don’t know what i..."
2031,2031,Birla-Institute-of-Technology-and-Science-Pila...,What are the chances of me getting an MSc. in ...,10,4,10/4,No,Followed,2018-07-21,https://www.quora.com/What-are-the-chances-of-...,Hey! Observing the last year's cutoffs for Msc...
2032,2032,Birla-Institute-of-Technology-and-Science-Pila...,What are the chances of me getting an MSc. in ...,10,4,10/4,No,Followed,2018-07-21,https://www.quora.com/What-are-the-chances-of-...,The chances are slim as the cutoff for the 4th...
2033,2033,Birla-Institute-of-Technology-and-Science-Pila...,What is BITS Pilani's cutoff after the 4th ite...,42,2,42/2,Yes,Followed,2018-07-20,https://www.quora.com/What-is-BITS-Pilanis-cut...,These are most accurate cutoff for 2019 so far...


In [4]:
df=df.reset_index()


In [5]:
df=df.drop_duplicates()

In [6]:
df=df.drop(['index'],axis=1)

In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in df['answers']]

In [8]:
#classifying sentiments as positive,negative and neutral
def sentiment(compound):
    res = ""
    if(compound >= 0.05):
        res="Positive"
    elif(compound <= - 0.05):
        res="Negative"
    else:
        res="Neutral"
    return res

df['Sentiment'] = df['compound'].apply(lambda x: sentiment(x))

In [9]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment
0,0,Birla-Institute-of-Technology-and-Science-Pila...,What made BITS Pilani so good? How did it mana...,2,2,2/2,No,Followed,2020-07-24,https://www.quora.com/What-made-BITS-Pilani-so...,Thanks for A2A What makes a college good in ge...,0.9928,Positive
1,1,Birla-Institute-of-Technology-and-Science-Pila...,What made BITS Pilani so good? How did it mana...,2,2,2/2,No,Followed,2020-07-24,https://www.quora.com/What-made-BITS-Pilani-so...,"Andhon mein kana, Raja ! This is the proverb t...",0.8397,Positive
2,2,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better, BITS Pilani Microelectronics ...",5,2,5/2,No,Followed,2020-07-20,https://www.quora.com/Which-is-better-BITS-Pil...,This correct answer is in Indian context: Sinc...,-0.9399,Negative
3,3,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better, BITS Pilani Microelectronics ...",5,2,5/2,No,Followed,2020-07-20,https://www.quora.com/Which-is-better-BITS-Pil...,Obviously BITS Microelectronics. BITS is the f...,0.8378,Positive
4,4,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better to get into, BITS Pilani DUBAI...",1,1,1/1,No,Followed,2020-07-19,https://www.quora.com/Which-is-better-to-get-i...,It's absolute foolishness to compare IITs with...,0.1282,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030,2030,Birla-Institute-of-Technology-and-Science-Pila...,Does it even make sense to expect any drop in ...,30,2,30/2,Yes,Followed,2018-07-26,https://www.quora.com/Does-it-even-make-sense-...,"Thanks for A2A!!!! Well, You don’t know what i...",0.8272,Positive
2031,2031,Birla-Institute-of-Technology-and-Science-Pila...,What are the chances of me getting an MSc. in ...,10,4,10/4,No,Followed,2018-07-21,https://www.quora.com/What-are-the-chances-of-...,Hey! Observing the last year's cutoffs for Msc...,0.9720,Positive
2032,2032,Birla-Institute-of-Technology-and-Science-Pila...,What are the chances of me getting an MSc. in ...,10,4,10/4,No,Followed,2018-07-21,https://www.quora.com/What-are-the-chances-of-...,The chances are slim as the cutoff for the 4th...,0.7003,Positive
2033,2033,Birla-Institute-of-Technology-and-Science-Pila...,What is BITS Pilani's cutoff after the 4th ite...,42,2,42/2,Yes,Followed,2018-07-20,https://www.quora.com/What-is-BITS-Pilanis-cut...,These are most accurate cutoff for 2019 so far...,0.1779,Positive


In [10]:
df['score'] = pd.cut(df['compound'], bins=5, labels=[1, 2, 3, 4, 5])
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score
0,0,Birla-Institute-of-Technology-and-Science-Pila...,What made BITS Pilani so good? How did it mana...,2,2,2/2,No,Followed,2020-07-24,https://www.quora.com/What-made-BITS-Pilani-so...,Thanks for A2A What makes a college good in ge...,0.9928,Positive,5
1,1,Birla-Institute-of-Technology-and-Science-Pila...,What made BITS Pilani so good? How did it mana...,2,2,2/2,No,Followed,2020-07-24,https://www.quora.com/What-made-BITS-Pilani-so...,"Andhon mein kana, Raja ! This is the proverb t...",0.8397,Positive,5
2,2,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better, BITS Pilani Microelectronics ...",5,2,5/2,No,Followed,2020-07-20,https://www.quora.com/Which-is-better-BITS-Pil...,This correct answer is in Indian context: Sinc...,-0.9399,Negative,1
3,3,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better, BITS Pilani Microelectronics ...",5,2,5/2,No,Followed,2020-07-20,https://www.quora.com/Which-is-better-BITS-Pil...,Obviously BITS Microelectronics. BITS is the f...,0.8378,Positive,5
4,4,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better to get into, BITS Pilani DUBAI...",1,1,1/1,No,Followed,2020-07-19,https://www.quora.com/Which-is-better-to-get-i...,It's absolute foolishness to compare IITs with...,0.1282,Positive,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030,2030,Birla-Institute-of-Technology-and-Science-Pila...,Does it even make sense to expect any drop in ...,30,2,30/2,Yes,Followed,2018-07-26,https://www.quora.com/Does-it-even-make-sense-...,"Thanks for A2A!!!! Well, You don’t know what i...",0.8272,Positive,5
2031,2031,Birla-Institute-of-Technology-and-Science-Pila...,What are the chances of me getting an MSc. in ...,10,4,10/4,No,Followed,2018-07-21,https://www.quora.com/What-are-the-chances-of-...,Hey! Observing the last year's cutoffs for Msc...,0.9720,Positive,5
2032,2032,Birla-Institute-of-Technology-and-Science-Pila...,What are the chances of me getting an MSc. in ...,10,4,10/4,No,Followed,2018-07-21,https://www.quora.com/What-are-the-chances-of-...,The chances are slim as the cutoff for the 4th...,0.7003,Positive,5
2033,2033,Birla-Institute-of-Technology-and-Science-Pila...,What is BITS Pilani's cutoff after the 4th ite...,42,2,42/2,Yes,Followed,2018-07-20,https://www.quora.com/What-is-BITS-Pilanis-cut...,These are most accurate cutoff for 2019 so far...,0.1779,Positive,3


In [11]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [12]:
df['followers_n']=normalize('Followers')

In [13]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score,followers_n
0,0,Birla-Institute-of-Technology-and-Science-Pila...,What made BITS Pilani so good? How did it mana...,2,2,2/2,No,Followed,2020-07-24,https://www.quora.com/What-made-BITS-Pilani-so...,Thanks for A2A What makes a college good in ge...,0.9928,Positive,5,0.011696
1,1,Birla-Institute-of-Technology-and-Science-Pila...,What made BITS Pilani so good? How did it mana...,2,2,2/2,No,Followed,2020-07-24,https://www.quora.com/What-made-BITS-Pilani-so...,"Andhon mein kana, Raja ! This is the proverb t...",0.8397,Positive,5,0.011696
2,2,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better, BITS Pilani Microelectronics ...",5,2,5/2,No,Followed,2020-07-20,https://www.quora.com/Which-is-better-BITS-Pil...,This correct answer is in Indian context: Sinc...,-0.9399,Negative,1,0.029240
3,3,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better, BITS Pilani Microelectronics ...",5,2,5/2,No,Followed,2020-07-20,https://www.quora.com/Which-is-better-BITS-Pil...,Obviously BITS Microelectronics. BITS is the f...,0.8378,Positive,5,0.029240
4,4,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better to get into, BITS Pilani DUBAI...",1,1,1/1,No,Followed,2020-07-19,https://www.quora.com/Which-is-better-to-get-i...,It's absolute foolishness to compare IITs with...,0.1282,Positive,3,0.005848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030,2030,Birla-Institute-of-Technology-and-Science-Pila...,Does it even make sense to expect any drop in ...,30,2,30/2,Yes,Followed,2018-07-26,https://www.quora.com/Does-it-even-make-sense-...,"Thanks for A2A!!!! Well, You don’t know what i...",0.8272,Positive,5,0.175439
2031,2031,Birla-Institute-of-Technology-and-Science-Pila...,What are the chances of me getting an MSc. in ...,10,4,10/4,No,Followed,2018-07-21,https://www.quora.com/What-are-the-chances-of-...,Hey! Observing the last year's cutoffs for Msc...,0.9720,Positive,5,0.058480
2032,2032,Birla-Institute-of-Technology-and-Science-Pila...,What are the chances of me getting an MSc. in ...,10,4,10/4,No,Followed,2018-07-21,https://www.quora.com/What-are-the-chances-of-...,The chances are slim as the cutoff for the 4th...,0.7003,Positive,5,0.058480
2033,2033,Birla-Institute-of-Technology-and-Science-Pila...,What is BITS Pilani's cutoff after the 4th ite...,42,2,42/2,Yes,Followed,2018-07-20,https://www.quora.com/What-is-BITS-Pilanis-cut...,These are most accurate cutoff for 2019 so far...,0.1779,Positive,3,0.245614


In [14]:
engagement_median = df['followers_n'].median()
df["engagement"] = df['followers_n'].apply(lambda x: 1 if x >= engagement_median else 0)

In [15]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [16]:
df['sentiment_score']=df['score']
df_eng1=df[df.engagement==1]
#if engagement score is greater than the median, increase the positive and negative sentiment
df_eng1['sentiment_score']=df_eng1['score'].apply(lambda x: assignpred(x))
df[df.engagement==1]=df_eng1
df

C:\Users\Kirthika\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score,followers_n,engagement,sentiment_score
0,0,Birla-Institute-of-Technology-and-Science-Pila...,What made BITS Pilani so good? How did it mana...,2,2,2/2,No,Followed,2020-07-24,https://www.quora.com/What-made-BITS-Pilani-so...,Thanks for A2A What makes a college good in ge...,0.9928,Positive,5,0.011696,0,5
1,1,Birla-Institute-of-Technology-and-Science-Pila...,What made BITS Pilani so good? How did it mana...,2,2,2/2,No,Followed,2020-07-24,https://www.quora.com/What-made-BITS-Pilani-so...,"Andhon mein kana, Raja ! This is the proverb t...",0.8397,Positive,5,0.011696,0,5
2,2,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better, BITS Pilani Microelectronics ...",5,2,5/2,No,Followed,2020-07-20,https://www.quora.com/Which-is-better-BITS-Pil...,This correct answer is in Indian context: Sinc...,-0.9399,Negative,1,0.029240,1,1
3,3,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better, BITS Pilani Microelectronics ...",5,2,5/2,No,Followed,2020-07-20,https://www.quora.com/Which-is-better-BITS-Pil...,Obviously BITS Microelectronics. BITS is the f...,0.8378,Positive,5,0.029240,1,5
4,4,Birla-Institute-of-Technology-and-Science-Pila...,"Which is better to get into, BITS Pilani DUBAI...",1,1,1/1,No,Followed,2020-07-19,https://www.quora.com/Which-is-better-to-get-i...,It's absolute foolishness to compare IITs with...,0.1282,Positive,3,0.005848,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030,2030,Birla-Institute-of-Technology-and-Science-Pila...,Does it even make sense to expect any drop in ...,30,2,30/2,Yes,Followed,2018-07-26,https://www.quora.com/Does-it-even-make-sense-...,"Thanks for A2A!!!! Well, You don’t know what i...",0.8272,Positive,5,0.175439,1,5
2031,2031,Birla-Institute-of-Technology-and-Science-Pila...,What are the chances of me getting an MSc. in ...,10,4,10/4,No,Followed,2018-07-21,https://www.quora.com/What-are-the-chances-of-...,Hey! Observing the last year's cutoffs for Msc...,0.9720,Positive,5,0.058480,1,5
2032,2032,Birla-Institute-of-Technology-and-Science-Pila...,What are the chances of me getting an MSc. in ...,10,4,10/4,No,Followed,2018-07-21,https://www.quora.com/What-are-the-chances-of-...,The chances are slim as the cutoff for the 4th...,0.7003,Positive,5,0.058480,1,5
2033,2033,Birla-Institute-of-Technology-and-Science-Pila...,What is BITS Pilani's cutoff after the 4th ite...,42,2,42/2,Yes,Followed,2018-07-20,https://www.quora.com/What-is-BITS-Pilanis-cut...,These are most accurate cutoff for 2019 so far...,0.1779,Positive,3,0.245614,1,3


In [17]:
df.rename(columns = {'score':'old_score'}, inplace = True) 

In [18]:
df.to_csv('../../Datasets/Quora/BITSPQuoraWSA.csv')